In [13]:
from scipy.io import loadmat
import os 
import re 
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
#Setting the folders in which to retrieve the data from and run the code
rootpath = "E:\Thesis work\Pendrive\Risk preferences in PD\Effort choices\Analysis"
data_location = os.path.join(rootpath,"PD_effort data for analysis")

In [3]:
#Lists all folders in the data folder
folders = os.listdir(data_location)

In [4]:
#Since the data was accrued on two days identified by the codes ON and OFF, making two lists containing the subject names
#in each of the folders. 
name_off=[];
name_on=[];

for item in folders:
    if re.search("^GR\w*\sOFF$",item) is not None:
        name_off.append(item)
    elif re.search("^GR\w*\sON$",item) is not None:
        name_on.append(item)

In [5]:
print(name_off)

['GR0001 OFF', 'GR0012 OFF', 'GR0065 OFF', 'GR0071 OFF', 'GR0097 OFF', 'GR0098 OFF', 'GR0099 OFF', 'GR0101 OFF', 'GR0102 OFF', 'GR0106 OFF', 'GR0108 OFF', 'GR0111 OFF', 'GR0180 OFF', 'GR0182 OFF', 'GR0184 OFF', 'GR0187 OFF', 'GR0195 OFF', 'GR0207 OFF', 'GR0208 OFF']


In [6]:
#List of lists of subject names
masterlist = list(map(list,zip(name_off,name_on)))

The next part of the code will iterate through the subject folders. In each folder,there are five folders each associated with a phase of the experiment: MVC, Association (assoc), Recall, Choice and Trial Selection. MVC, Assoc and Recall involved grip force exertion and have force recording values. 

One master data frame will be made containing the force recordings for every trial from all these phases across trials, subjects and treatment days. 

In [7]:
Volt = np.empty([19*2*(3+40+48),8002])
MVCPhase = np.empty([19*2,3])
AssocPhase = np.empty([19*2*(40),3])
RecallPhase = np.empty([19*2*(48),5])
count=0

In [8]:
for i in range(len(masterlist)):
    for j in range(len(masterlist[i])):
        #Load MVC data with necessary calculations
        folder = os.path.join(data_location,masterlist[i][j],"MVCPhase.mat")
        mvc = loadmat(folder)
        
        a = np.tile([i+1,j+1],[3,1])
        
        Volt[count:count+3,:] = np.concatenate((mvc['voltMVCTrial'],a),axis=1)
        count=count+3
        
        folder = os.path.join(data_location,masterlist[i][j],"AssocPhase.mat")
        assoc = loadmat(folder)
        
        a = np.tile([i+1,j+1],[40,1])
        Volt[count:count+40,:] = np.concatenate((assoc['voltAssocTrial'],a),axis=1)
        count = count+40
        
        folder = os.path.join(data_location,masterlist[i][j],"RecallPhase.mat")
        recall = loadmat(folder)
        
        a = np.tile([i+1,j+1],[48,1])
        Volt[count:count+48,:] = np.concatenate((recall['voltRecallTrial'],a),axis=1)
        count = count+48
        
        MVCPhase[2*i+j,:] = [i+1,j+1,0.8*np.max(np.array(mvc['voltMVCTrial']))]
        AssocPhase[(2*i+j)*40:(2*i+j+1)*40,0:3] = np.concatenate((np.array(assoc['AssocTrial'][:,0:2]),np.tile(MVCPhase[2*i+j,2],[40,1])),axis =1)
        RecallPhase[(2*i+j)*48:(2*i+j+1)*48,0:5] = np.concatenate((np.array(recall['RecallTrial'][:,0:4]),np.tile(MVCPhase[2*i+j,2],[48,1])),axis =1)


In [9]:
Volts = pd.DataFrame(Volt)
Volts.rename(columns = {8000:'SID',8001:'TREAT'},inplace=True)

#Add a variable to describe the trial number.          
TrialNum = np.concatenate((np.arange(1,4),np.arange(1,41),np.arange(1,49)))
Volts['TrialNum']  = np.reshape(np.tile(TrialNum,[38,1]),91*38)   

#Add a variable to describe which phase the data is from: 1 for MVC, 2 for Assoc and 3 for Recall
PhaseVar = np.concatenate((np.tile(1,[3,1]),np.tile(2,[40,1]),np.tile(3,[48,1])))
Volts['PhaseVar'] =  np.tile(PhaseVar,[38,1])
 
Volts['Mean'] = Volts.iloc[:,2000:7998].mean(axis=1)    #Mean force exerted over 4s
Volts['Std'] = Volts.iloc[:,2000:7998].std(axis=1)      #Standard deviation in force exerted
Volts['MaxVolt'] = Volts.iloc[:,0:7999].max(axis=1)     #Maximum force exerted during the 4s

#Change identifier variables to categorical. Will help with linear regressions later. 
Volts[['SID','TREAT','TrialNum','PhaseVar']]= Volts[['SID','TREAT','TrialNum','PhaseVar']].astype('category')

In [10]:
#Populating the association dataframe. The dataframe has the target force, Outcome (success or failure) and Max force. 
AssocPhase = pd.DataFrame(AssocPhase,columns = ['T','O','MVC'])

#Adding subject ID, Treatment condition, Trial number, mean exertion and standard deviation
AssocPhase[['SID','TREAT','TrialNum','M','S']]= Volts[Volts['PhaseVar']==2].loc[:,['SID','TREAT','TrialNum','Mean','Std']].reset_index(drop=True)
#Mean and Standard deviation are normalised to individual maximum grip force
AssocPhase.loc[:,['M','S']] = AssocPhase.loc[:,['M','S']].div(AssocPhase['MVC'],axis=0)*100
#A normalised metric of variability is defined. 
AssocPhase['SDN']=AssocPhase['S']/AssocPhase['M']

In [12]:
#Populating the recall dataframe. The dataframe has the target force,reported effort level,
#Reaction time to report, Outcome (success or failure) and Max force.
RecallPhase = pd.DataFrame(RecallPhase,columns = ['T','R','RT','O','MVC'])
#Adding subject ID, Treatment condition, Trial number, mean exertion and standard deviation
RecallPhase[['SID','TREAT','TrialNum','M','S']]= Volts[Volts['PhaseVar']==3].loc[:,['SID','TREAT','TrialNum','Mean','Std']].reset_index(drop=True)
#Mean and Standard deviation are normalised to individual maximum grip force
RecallPhase[['M','S']] = RecallPhase[['M','S']].div(RecallPhase['MVC'],axis=0)*100
#A normalised metric of variability is defined. 
RecallPhase['SDN']=RecallPhase['S']/RecallPhase['M']
#RM - Difference between report and exerted effort. To determine accuracy of perceiving exerted effort. 
RecallPhase['RM']= RecallPhase['R'] - RecallPhase['M']
RecallPhase['RMabs'] = np.absolute(RecallPhase['RM'])

The data is now processed and all the variables required for further analysis are available. 
Goal: 1. To determine whether acuity of effort perception is affected by medication. 
2. If not, can we outline a mechanism to explain that. 

In [ ]:
#Save all the dataframes into csv files that can be loaded up for further analysis. Once the data is set-up,
#we should not have to run this code again. 



In [14]:
mixed = smf.mixedlm("R ~ M*TREAT", RecallPhase, groups = RecallPhase["SID"])
mixed_fit = mixed.fit()
#print the summary
print(mixed_fit.summary())

          Mixed Linear Model Regression Results
Model:             MixedLM Dependent Variable: R         
No. Observations:  1824    Method:             REML      
No. Groups:        19      Scale:              261.6134  
Min. group size:   96      Likelihood:         -7696.5521
Max. group size:   96      Converged:          Yes       
Mean group size:   96.0                                  
---------------------------------------------------------
               Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------
Intercept       1.529    2.275  0.672 0.502 -2.931  5.988
TREAT[T.2.0]    4.877    1.865  2.615 0.009  1.222  8.532
M               1.216    0.031 39.376 0.000  1.155  1.276
M:TREAT[T.2.0] -0.189    0.042 -4.473 0.000 -0.272 -0.106
Group Var      64.358    1.389                           



In [17]:
mixed2 = smf.mixedlm("S ~ M*TREAT", RecallPhase, groups = RecallPhase["SID"])
mixed_fit2 = mixed2.fit()
#print the summary
print(mixed_fit2.summary())

          Mixed Linear Model Regression Results
Model:             MixedLM Dependent Variable: S         
No. Observations:  1824    Method:             REML      
No. Groups:        19      Scale:              16.7453   
Min. group size:   96      Likelihood:         -5186.8449
Max. group size:   96      Converged:          Yes       
Mean group size:   96.0                                  
---------------------------------------------------------
               Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------
Intercept       0.834    0.441  1.889 0.059 -0.031  1.699
TREAT[T.2.0]    1.388    0.472  2.942 0.003  0.463  2.312
M               0.206    0.008 26.330 0.000  0.190  0.221
M:TREAT[T.2.0] -0.037    0.011 -3.477 0.001 -0.058 -0.016
Group Var       1.525    0.139                           



In [19]:
mixed3 = smf.mixedlm("RM ~ SDN*TREAT", RecallPhase, groups = RecallPhase["SID"])
mixed_fit3 = mixed3.fit()
#print the summary
print(mixed_fit3.summary())

           Mixed Linear Model Regression Results
Model:              MixedLM  Dependent Variable:  RM        
No. Observations:   1824     Method:              REML      
No. Groups:         19       Scale:               264.3227  
Min. group size:    96       Likelihood:          -7695.6009
Max. group size:    96       Converged:           Yes       
Mean group size:    96.0                                    
------------------------------------------------------------
                 Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
------------------------------------------------------------
Intercept         5.320    2.212  2.405 0.016   0.984  9.656
TREAT[T.2.0]     -0.989    1.604 -0.617 0.538  -4.132  2.155
SDN              20.348    4.998  4.071 0.000  10.552 30.144
SDN:TREAT[T.2.0] -7.332    5.938 -1.235 0.217 -18.971  4.306
Group Var        61.260    1.319                            



In [21]:
mixed4 = smf.mixedlm("RMabs ~ SDN*TREAT", RecallPhase, groups = RecallPhase["SID"])
mixed_fit4 = mixed4.fit()
#print the summary
print(mixed_fit4.summary())

           Mixed Linear Model Regression Results
Model:              MixedLM  Dependent Variable:  RMabs     
No. Observations:   1824     Method:              REML      
No. Groups:         19       Scale:               134.4276  
Min. group size:    96       Likelihood:          -7079.2498
Max. group size:    96       Converged:           Yes       
Mean group size:    96.0                                    
------------------------------------------------------------
                 Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
------------------------------------------------------------
Intercept        12.692    1.516  8.371 0.000   9.721 15.664
TREAT[T.2.0]      1.500    1.144  1.312 0.190  -0.741  3.742
SDN              13.684    3.564  3.839 0.000   6.699 20.670
SDN:TREAT[T.2.0] -8.980    4.234 -2.121 0.034 -17.279 -0.681
Group Var        27.540    0.837                            

